# Energy Storage Project

In the next picture it is shown the configuration of the system as well as the different variables, as a first step we need to carry on an energy balance that must be satisfied. It is presented in the next equation:

<img width="400" src="system.jpg">


$\displaystyle \quad f_w[i] P_w+f_s[i] P_s + \alpha_d[i] P_d + \alpha_{nd}[i] P_{nd} = P_{dem}[i] + \alpha_b[i] P_b + P_{un} [i]$

Just as a note when the variables or constants have an index $[i]$, it represents that this variable correspond to the $i$ period. It will be one equation for each period.

An other important characteristic to model for the systems is the battery. It must be taken into account that the battery can be charged and discharged, changing the capacity. We can define a new variable $E_{max,b}$ which corresponds to the maximal energy that can be stored in the battery in MWh. Doing an energy balance for the battery: the energy stored in the $i+1$ period will be equal to the energy in the battery in the $i$ period plus the energy received or transferred from the battery, i.e.,

$\quad \displaystyle E_{b}[i+1] = E_{b}[i] + \alpha_{b}[i] P_b \Delta t$ 

Note that $-1 \leq \alpha_{b}[i] \leq 1$. When $\alpha_{b}[i]$ is positive it means that the baterry has been charged, when it is negative the batery has been discharged. To limit the charging and discharging process in function of the maximal capacity of the battery, the last equation is divided by $E_{max,b}$:

$\quad \displaystyle \frac{E_{b}[i+1]}{E_{max,b}} = \frac{E_b[i]}{E_{max,b}} + \frac{\alpha_{b}[i]P_b\Delta t}{E_{max,b}}$

It can be defined a new variable that is commonly founded in the literature when batteries are modelated, which is called State of Charge ($SOC$) defined as follows:

$\quad \displaystyle SOC[i]=\frac{E_b[i]}{E_{max,b}}$

also due to the fact that $\Delta t = 1 \text{ h}$ we can do not consider this term obtaining the next expression:

$\quad \displaystyle SOC[i+1]=SOC[i]+\alpha_b[i]\frac{P_b}{E_{max,b}}$

Then, it is needed to consider the next restriction:

$\quad \displaystyle 0 \leq SOC[i]\leq 1$

Next we present all the constant and variables used until now in the model proposed. 
##### Constants

| notation | meaning | units |
| -: | :- | :-: |
| $f_w[i]$ | nominal production of wind energy during the $i$ period. | - |
| $f_s[i]$ | nominal prodcution of solar energy during the $i$ period.| - |
| $P_w$ | maximal capacity of the wind plant.| MW |
| $P_s$ | maximal capacity of the solar plant.| MW |
| $P_d$ | maximal capacity of the old-diesel plant.| MW |
| $P_{dem}[i]$ | power demand during the $i$ period | MW |

##### Variables

| notation | meaning | units |
| -: | :- | :-: |
| $\alpha_d[i]$ | operating capacity factor of the old-diesel plant during the $i$ period | - |
| $\alpha_{nd}[i]$ | operating capacity factor of the new-diesel plant during the $i$ period | - |
| $P_{nd}$ | maximal capacity required of the new-diesel plant | MW |
| $\alpha_b[i]$ | operating capacity factor of the battery during the $i$ period | - |
| $P_{b}$ | maximal capacity required for the battery | MW |
| $P_{un}[i]$ | loss load during the $i$ period | MW |
| $SOC[i]$ | state of charge of the battery during the $i$ period | - |
| $E_{max,b}$ | maximal energy capacity of the battery | MWh |

To restring the factible set of the problem, there will be some restrictions in the variables, next we present all of thems:

$\quad 0 \leq \alpha_d[i] \leq 1$

$\quad 0 \leq \alpha_{nd}[i] \leq 1$

$\quad P_{nd} \geq 0$

$\quad-1 \leq \alpha_b[i] \leq 1$

$\quad P_b \geq 0$

$\quad P_{un}[i] \geq 0$

$\quad 0 \leq SOC[i] \leq 1$

$\quad E_{max,b} \geq 0$

Until now, just the system has been modeled. However, to carry on the optimization problem we need to define an objective function which depends on three factors: 1) the investment cost, 2) the operation and maintenance cost and 3) a variable cost. First, let's consider just the variable cost. Considering the different componentes has been taken into account, just a variable cost can be associated to the diesel generation and to the non-served energy:

$\displaystyle \quad C_{var} = \sum_{i=1} ^{N} \left[ \alpha_d[i]P_d C_d + \alpha_{nd}[i]P_{nd}C_d + P_{un}[i]C_{un}\right]  \Delta t$

where $N$ is the number of periods considered in the optimization period, $C_d$ is the unitary cost of diesel in €/MWh and $C_{un}$ the unitary cost of the unserved energy in €/MWh.